In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import sys
import re
import logging
import os
from pathlib import Path
from functools import partial
from itertools import chain
from uuid import uuid4

import blpapi
import ipywidgets as w
import ipyvuetify as v
import numpy as np
import pandas as pd
import QuantLib as ql
from IPython.display import display
from ipydatagrid import DataGrid, TextRenderer, BarRenderer, Expr, VegaExpr
from bqplot import LinearScale, ColorScale, OrdinalColorScale, OrdinalScale
from pandas.tseries.offsets import BDay
from plotly import graph_objects as go
from plotly.subplots import make_subplots

import bhutils
from bhutils import bhutilspy as bh
# from api.ezutils import bh, mkt_defs, ccy_cals, mx_cals, mtg_dts, gens

from gioutils.utils import parse_offset, today, get_next_n
from api.data.base import data_path, radar_path
from gioutils.gui.base import View, store, WidgetView, Tabs
from api.gui.params import curves, bbg_params, spot_params, fwd_params
from gioutils.blpw import BlpQuery
from gioutils.utils import RepeatedTimer
from gioutils.ezutils import bh, bhs
from api.gui.theme import bg_color, grid_style
from api.gui.pandas import DFOutput, negative_red, custom_styler_row_line, DataGridGio

pd.options.plotting.backend = "plotly"

In [3]:
from api.gui.collectors import DataCollector, SpotForward, Futures, Market, Mtgs, FwdMatrix
from api.gui.fly_grids import EditableGrid
from api.gui.tables import SpotTable, FutureTable, GroupMktTalbe, KeyChart, MtgTable, MatTable, CustomGovtBondFutTable
from api.gui.grids import SpotGrid, FutureGrid, MtgGrid, MatGrid, GroupMktGrid # GroupMktTalbe, KeyChart, MtgTable, MatTable, CustomGovtBondFutTable
from api.gui.global_config import GlobalConfig
from api.gui.market_monitor import Monitor, Monitor2

In [4]:
bq = BlpQuery(timeout=50000).start()

In [5]:
ccys = [
    'EUR', 
    'USD', 
    'GBP'
]

In [6]:
curves_df = pd.DataFrame(curves).set_index('ccy').loc[ccys]

In [7]:
fly_grid = EditableGrid()

In [8]:
m = Monitor2()

In [9]:
dc = DataCollector(
    curves=curves_df.melt()['value'].drop_duplicates().dropna().to_list(),
    bq=bq,
    collectors={
        'eg': fly_grid,
        'sf': SpotForward(spot_params=spot_params, fwd_params=fwd_params),
        'fut': Futures(),
        'mkt': Market(),
        'mtgs': Mtgs(),
        'mat': FwdMatrix()
    },
    renderers={
        'monitor': m,
    }
)

In [10]:
gc = GlobalConfig(dc=dc)

In [11]:
tabs = Tabs(
    {
        'radar': v.Container(
            fluid=True,
            children=[
                v.Row(children=[v.Col(children=[gc.view])]),
                v.Row(children=[v.Col(children=[m.view])])
            ],
        ),
        'flies': v.Container(
            fluid=True,
            children=[
                fly_grid.param_box,
                v.Row(children=[v.Col(children=[fly_grid.dg])])
            ],
        ),
        'watchlist': fly_grid.expr_dg.view,
    },
    force_grid_render=True,
    # tabs_kwargs={'class_': "mb-2"}
)
tabs

Card(children=[Tabs(children=[Tab(children=['radar']), Tab(children=['flies']), Tab(children=['watchlist'])], …

In [ ]:
from gioutils.ezutils import LiveCurvesRemote, LiveVolsRemote, LiveInflLocal, get_id
lc = LiveCurvesRemote()
lv = LiveVolsRemote()
li = LiveInflLocal()